# Estimator
_________________________________________________________________
## 에스티메이터는 딥러닝 모델을 최대한 빠르게 실험해보고 다양한 모델을 적용할 수 있게 해줘서 모델 구현에 집중할 수 있는 환경을 제공한다.
## 에스티메이터는 고수준 API를 통해 이러한 환경을 제공한다.
## 모델 구현 외의 학습(Train), 검증(Evaluate), 예측(Predict)에 필요한 부가적인 구현들은 함수를 통해 제공한다.
## 모든 검증과 평가가 끝난 모델은 배포(Export)모델을 통해 구현할 수 있다.

____________________________________________________________________________________________________________________________
### 에스티메이터에서 제공하는 기본적인 기능
#### - 학습(Train) : 정의한 모델 파라미터에 대해 학습한다.
#### - 평가(Evaluate): 학습한 모델에 대한 성능을 측정한다.
#### - 예측(Predict): 모델을 통해 입력값에 대한 예측값을 받는다.
#### - 배포(Export): 사용할 모델을 바이너리 파일로 출력한다.

___________________________________________________________________________________________________________________________
#### 에스티메이터를 구현하기 위해서는 기본적으로 model_fn()과 train_input_fn()을 구현해야한다.
#### model함수의 인자는 총 다섯개이며 다음과같다.
##### - features: 모델에 적용되는 입력값, 학습, 검증, 예측 과정 모두에 사용된다.
##### - labels: 모델의 정답 라벨값을 의미한다., 예측 과정에서는 라벨이 존재하지 않기 떄문에 에스티메이터에서 자동으로 이 값이 들어오지 않는다
##### - mode: 현재 모델 함수가 실행된 모드(학습, 검증, 예측)를 의미한다.
##### - params: 모델에 적용될 부가적인 하이퍼파라미터 값을 의미한다.
##### - config: 모델에 적용할 설정값을 의미한다.
#### 이중 features와 labels는 필수적으로 인자로 받도록 설정, 나머지는 필요에따라 선택적으로 받을 수 있다.

In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow.keras import preprocessing

tf.disable_v2_behavior()

In [2]:
# 데이터는 이전 장에서 사용했던 텍스트 데이터를 그대로 사용
# 전처리 과정 또한 동일하게 적용
samples = ['너 오늘 이뻐 보인다', 
           '나는 오늘 기분이 더러워', 
           '끝내주는데, 좋은 일이 있나봐', 
           '나 좋은 일이 생겼어', 
           '아 오늘 진짜 짜증나', 
           '환상적인데, 정말 좋은거 같아']

labels = [[1], [0], [1], [1], [0], [1]]

In [3]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

In [4]:
# 입력 함수에 필요한 에폭 값
EPOCH = 100

# 데이터를 에스티메이터에 전달하기 위한 데이터 입력 함수
# tf.data의 반복, 셔플 기능을 통해 입력 함수를 구현
# 정의한 에폭 크기만큼 데이터를 반복시켜서 학습하게 하고, 셔플기능을 통해 모델이 학습을 잘 할 수 있게 한다.
# 함수 반환값은 이터레이터의 get_next함수를 사용하여 입력값과 라벨값을 반환한다.
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    dataset = dataset.repeat(EPOCH)
    dataset = dataset.batch(1)
    dataset = dataset.shuffle(len(sequences))
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [5]:
# 모델 함수에 필요한 단어 개수와 임베딩 벡터 크기의 상숫값
VOCAB_SIZE = len(word_index) +1
EMB_SIZE = 128

In [6]:
# 사용할 모델을 구현한 모델함수
# 이를 Estimator 객체에 적용하면 된다.
# 이를 통해 생성한 객체는 학습, 검증, 예측을 사용할 수 있다.

# 인자로 입력값과 라벨, 그리고 현재 모델 함수가 실행되는 상태인 모드값을 받는다.
def model_fn(features, labels, mode):
    
    # TRAIN, EVALUATE, PREDICT상태 중 해당하는 상태의 상숫값이 True가 된다.
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    # 입력값을 임베딩 벡터 형태로 만들어준다
    embed_input = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features)
    # 텐서플로의 reduce_mean 함수를 통해 평균을 구해서 하나의 입력 벡터로 만들어준다.
    embed_input = tf.reduce_mean(embed_input, axis=-1)
    
    # 이 입력값을 Dense를 사용해 은닉층을 거쳐 출력값을 만든다.
    hidden_layer = tf.keras.layers.Dense(128, activation=tf.nn.relu)(embed_input)
    output_layer = tf.keras.layers.Dense(1)(hidden_layer)
    # 시그모이드 함수를 적용해 0과 1사이의 값으로 만든다
    # 이것이 모델의 최종 출력값이다
    output = tf.nn.sigmoid(output_layer)
    
    # 학습 상태일 때 모델을 학습할 수 있도록 손실(loss)값과 옵티마이저(optimizer)를 설정해야한다.
    
    # 손실 함수는 평균 제곱 오차를 이용
    loss = tf.losses.mean_squared_error(output, labels)

    # 옵티마이저는 아담 옵티마이저 사용
    # 두 값을 학습 모드인경우 EstimatorSpec의 인자갑으로 반환한다.
    if TRAIN:
        global_step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)
        
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  train_op=train_op,
                  loss=loss)

In [7]:
# 실제 학습

# 학습 과정을 체크포인트로 저장할 폴더를 만들어준다.
DATA_OUT_PATH = './data_out/'

# os라이브러리를 사용해서 폴더의 유무를 확인한 후 존재하지 않는다면 폴더를 만들어준다.
import os

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# 앞서 정의한 모델 함수와 체크포인트를 저장할 폴더를 인자로 넣어준다.
estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = DATA_OUT_PATH + 'checkpoint/dnn')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
estimator.train(train_input_fn)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into ./data_out/checkpoint/dnn\model.ckpt.
INF